In [1]:
from instruct_tri2tri.tsr.system import InstructTri2Tri, TSR
from torch.utils.data import Dataset
import torch

class InstructTri2TriDataset(Dataset):
    def __init__(self,
                 data_path):
        super().__init__()
        datas = json.load(open(data_path))[:200]
        self.datas = datas

    def __len__(self):
        return len(self.datas)
    
    def __getitem__(self, index):
        data = self.datas[index]
        image_name = data['image']
        instruct_image_name = data['instruct_image']
        image = Image.open(f'data/{image_name}')
        instruct_image = Image.open(f'data/{instruct_image_name}')
        # image_name = image_name.split('/')[-1]
        instruct = data['instruct']
        return image, instruct_image, instruct


/root/data/anaconda3/envs/pixellm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tsr = TSR.from_pretrained(
        'stabilityai/TripoSR',
        'config.yaml',
        'model.ckpt'
    ).cuda()

In [2]:
model = InstructTri2Tri.from_pretrained(
    'instruct_tri2tri/tsr/instruct_tri2tri_config',
    config_name="config.yaml",
    weight_name="model.ckpt",
).cuda()
dataset = InstructTri2TriDataset('data/objaverse/cap3d_automated_objaverse_highquality_instruct_550k.json')

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.70 GiB total capacity; 1.64 GiB already allocated; 9.56 MiB free; 1.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [1]:
from transformers import CLIPTextModelWithProjection

clip = CLIPTextModelWithProjection.from_pretrained('ckpts/clip-vit-large-patch14').cuda()

/root/data/anaconda3/envs/pixellm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
